## 5.2 Modelos de Regresión

En esta clase vamos a centrarnos en los modelos de regresión, como explicamos en la clase anterior los modelos de regresión son aquellos modelos en los que su objetivo es predecir el valor de una variable numérica.

Nos centraremos en el modelo de regresión mas simple que es la regresión y posteriormente veremos algún modelo más complejo.

### 1. Regresión lineal

Para poder aplicar una regresión lineal a nuestros datos se han de tener algunas suposiciones previas:

+ Exogeneidad débil (predictores libres de error)
+ Linealidad
+ Homocedasticidad: La homocedasticidad es una característica de un modelo de regresión lineal que implica que la varianza de los errores es constante a lo largo del tiempo.
+ Independencia de los errores
+ Falta de colinealidad (independencia lineal)

$$y=\beta_0+\beta_1x_1+\beta_2x_2+\beta_3x_3+\ldots+\beta_nx_n+\epsilon$$

El objetivo de la regresion lineal es obtener los $\beta$:
+ Algebraicamente:

$$\vec{\beta} = (X^{T}X)^{-1}X^{T}y$$

+ Minimos cuadrados

$$MSE = \frac{1}{n} \sum_{i=1}^{n} (y-\hat{y})^{2}$$

Ejemplo)

$y = \beta_0 + \beta_1x_1 + \epsilon$

Entonces, derivando MSE e igualando a 0:

$\beta_0 = \frac{\sum y - \beta_1\sum x}{n} = \bar{y} - \beta_1\bar{x}$

$\beta_1=\frac{\sum (x-\hat{x})(y-\hat{y})}{\sum (x-\hat{x})}$

Vamos a ver esto en forma de código, para ello importamos las librerías necesarias

In [ ]:
#!pip install -U statsmodels #para actualizar la librería de statsmodels

In [ ]:
import pandas as pd 
import numpy as np

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

from statsmodels.api import add_constant, OLS
from statsmodels.formula.api import ols

import pylab as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

**Regularización**

Vamos a ver que es cada una de las librerías que hemos importado, cada una de ellas son diferentes métodos de regularización de una regresión lineal.

**Función de Coste o de Pérdida (J)** = Función a minimizar

En el caso de la regresión lineal: 
$$J=MSE$$


La regularización es una medida/penalización de la complejidad del modelo. Se añade un término a J que depende del tipo de regularización:

$$J = MSE + \alpha · T$$


+ Lasso (L1, norma 1):

$$T=\frac{1}{n}\sum_{i}  |\beta_i|$$

Muy útil si se sospecha que hay características irrelevantes. Se favorece $\beta \approx 0$

+ Ridge (L2):

$$T=\frac{1}{2n}\sum_{i}  \beta_{i}^{2}$$

Muy útil si se sospecha que existe correlación entre las características, minimiza esa correlación. Funciona mejor si todas son relevantes.

+ ElasticNet (L1+L2):

$$T=r·L1 + (1-r)·L2$$

Se usa cuando hay muchas características.

Vamos a cargar unos datos procedentes de las bases de datos que contiene sklearn y vamos a aplicale los diferentes modelos de regresión que hemos visto.

#### 1.1 Cargamos los datos

In [ ]:
boston=load_boston()

In [ ]:
boston

In [ ]:
print(boston['DESCR'])

In [ ]:
df=pd.DataFrame(boston.data, columns=boston.feature_names)

df['price']=boston.target

df.head()

#### 1.2 Exploración de los datos

##### Búsqueda de valores nulos

In [ ]:
df.info()

##### Descripción estadística de los datos

In [ ]:
df.describe().T

#### 1.3 Análisis de correlación y elección de variables

In [ ]:
plt.figure(figsize=(15, 10))

sns.set(style='white')

mask=np.triu(np.ones_like(df.corr(), dtype=bool))

cmap=sns.diverging_palette(0, 10, as_cmap=True)


sns.heatmap(df.corr(),
           mask=mask,
          cmap=cmap,
          center=0,
          square=True,
          annot=True,
          linewidths=0.5,
          cbar_kws={'shrink': 0.5});

Con ayuda de la librería de statsmodels y la técnica de ols (mínimos cuadrados) podemos ver la importancia de cada una de las variables independientes de nuestros datos respecto a la variable dependiente

Vamos a crear una función que recibe como parámetros la variable dependiente(y) y la variable independiente(x)

In [ ]:
def plot_regression_model(x,y):
    
    x_const = add_constant(x) # add a constant to the model
    
    modelo = OLS(y, x_const).fit() # fit the model
    
    pred = modelo.predict(x_const) # make predictions
    
    print(modelo.summary());
    try:
        const = modelo.params[0] # create a variable with the value of the constant given by the summary
        coef = modelo.params[1] # create a variable with the value of the coef given by the summary

        x_l=np.linspace(x.min(), x.max(), 50) 
        y_l= coef*x_l + const # function of the line

        plt.figure(figsize=(10, 10));

        # plot the line
        plt.plot(x_l, y_l, label=f'{x.name} vs {y.name}={coef}*{x.name}+{const}');

        # data
        plt.scatter(x, y, marker='x', c='g', label=f'{x.name} vs {y.name}');

        plt.title('Regresion lineal')
        plt.xlabel(f'{x.name}')
        plt.ylabel(f'{y.name}')
        plt.legend()
        plt.show();
        return modelo
    except:
        print('No se puede imprimir la recta de regresión para modelos multivariable')
        plt.show();
        return modelo

Separamos nuestras variables independientes de nuestra variable dependiente

In [ ]:
X=df.drop('price', axis=1)

y=df.price

In [ ]:
for c in X:
    plot_regression_model(X[c], y)

Como podemos ver predecir el precio de una casa a través de una regresión lineal con una única variable es bastante complicado, vamos a ver como se comportaría de forma multivariable

In [ ]:
multi = plot_regression_model(X,y)

Si nos fijamos hay algunas variables en el summary en el que la columna P nos indica 0, está P se está refiendo al pvalue, este valor no indica el estadístico de la variable independiente respecto de la dependiente, cuando más cercano a 0 esté nos indica que la variable es dependiente la una de la otra.
Esto se basa en el testeo de hipótesis, para ello determinamos nuestra hipótesis inicial que es que las variables son independientes y establecemos un intervalo de confianza sobre el que apoyar nuestra hipótesis, normalmente es del 95%, esto quiere decir que si el pvalue es menor de 0.05 podremos descartar nuestra hipótesis inicial y determinar con un intervalo de confianza del 95% que la variable dependiente realmente depende de la independiente

Y los betas (pesos) de nuestra ecuación correspondería a la columna coef. Así quedaría la ecuación de cada una de las rectas de regresión

$$ y = coef * x + const $$

Del mismo modo prodríamos obtener estos valores con la librería de sklearn

In [ ]:
ln = LinearRegression()

In [ ]:
ln.fit(X,y)

Coeficientes

In [ ]:
ln.coef_

In [ ]:
dict(zip(X.columns, ln.coef_))

Ordenada en el origen

In [ ]:
ln.intercept_

Una vez elegidas nuestras variables podemos pasar a realizar el train_test_split para después standarizar nuestros datos y entrenar nuestro modelo

In [ ]:
X = X.drop(['INDUS', 'AGE'], axis=1)

#### 1.4 Train test split y estandarización

In [ ]:
X_train ,X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=.2)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
sc = StandardScaler().fit(X_train)

In [ ]:
X_train_sc = sc.transform(X_train)
X_test_sc = sc.transform(X_test)

#### 1.5 Entrenamiento de modelos

Primero declaramos los modelos

In [ ]:
linreg=LinearRegression()
lasso=Lasso() # Favorece si nuestras betas son próximas a 0 L1
ridge=Ridge() # Favorece cuando hay sospechas de correlación entre caracteristicas L2
elastic=ElasticNet() # Mezcla de las anteriores, funciona bien cuando hay muchas características (L1+L2)

Entrenamos los modelos

In [ ]:
linreg.fit(X_train_sc, y_train)
lasso.fit(X_train_sc, y_train)
ridge.fit(X_train_sc, y_train)
elastic.fit(X_train_sc, y_train)

#### 1.6 Evaluación de modelos

##### **MSE**

$$MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{y}_i)^{2}$$


pertenece al intervalo [0, +$\infty$)

In [ ]:
y.describe()

In [ ]:
from sklearn.metrics import mean_squared_error as mse

print(f' MSE linreg en train: {mse(y_train, linreg.predict(X_train_sc))}\n')
print(f' MSE linreg en test: {mse(y_test, linreg.predict(X_test_sc))}\n')
print(f' MSE lasso en train: {mse(y_train, lasso.predict(X_train_sc))}\n')
print(f' MSE lasso en test: {mse(y_test, lasso.predict(X_test_sc))}\n')
print(f' MSE ridge en train: {mse(y_train, ridge.predict(X_train_sc))}\n')
print(f' MSE ridge en test: {mse(y_test, ridge.predict(X_test_sc))}\n')
print(f' MSE ElasticNet en train: {mse(y_train, elastic.predict(X_train_sc))}\n')
print(f' MSE ElasticNet en test: {mse(y_test, elastic.predict(X_test_sc))}\n')

##### **RMSE**



$$RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{y}_i)^{2}}$$


pertenece al intervalo [0, +$\infty$)

In [ ]:
print(f' RMSE linreg en train: {mse(y_train, linreg.predict(X_train_sc), squared=False)}\n')
print(f' RMSE linreg en test: {mse(y_test, linreg.predict(X_test_sc), squared=False)}\n')
print(f' RMSE lasso en train: {mse(y_train, lasso.predict(X_train_sc), squared=False)}\n')
print(f' RMSE lasso en test: {mse(y_test, lasso.predict(X_test_sc), squared=False)}\n')
print(f' RMSE ridge en train: {mse(y_train, ridge.predict(X_train_sc), squared=False)}\n')
print(f' RMSE ridge en test: {mse(y_test, ridge.predict(X_test_sc), squared=False)}\n')
print(f' RMSE ElasticNet en train: {mse(y_train, elastic.predict(X_train_sc), squared=False)}\n')
print(f' RMSE ElasticNet en test: {mse(y_test, elastic.predict(X_test_sc), squared=False)}\n')

##### **MAE**

$$MAE = \frac{1}{n}\sum_{i=1}^{n}|y_i-\hat{y}_i|$$


pertenece al intervalo [0, +$\infty$)

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

print(f' MAE linreg en train: {mae(y_train, linreg.predict(X_train_sc))}\n')
print(f' MAE linreg en test: {mae(y_test, linreg.predict(X_test_sc))}\n')
print(f' MAE lasso en train: {mae(y_train, lasso.predict(X_train_sc))}\n')
print(f' MAE lasso en test: {mae(y_test, lasso.predict(X_test_sc))}\n')
print(f' MAE ridge en train: {mae(y_train, ridge.predict(X_train_sc))}\n')
print(f' MAE ridge en test: {mae(y_test, ridge.predict(X_test_sc))}\n')
print(f' MAE ElasticNet en train: {mae(y_train, elastic.predict(X_train_sc))}\n')
print(f' MAE ElasticNet en test: {mae(y_test, elastic.predict(X_test_sc))}\n')

##### **R2 score**

$$R2 = 1 - \frac{\sum_{i=1}^{n}(y_i-\hat{y}_i)^{2}}{\sum_{i=1}^{n}(y_i-\bar{y})^{2}}$$

pertenecen al intervalo (-$\infty$, 1]

In [ ]:
print(f' R2 linreg en train: {linreg.score(X_train_sc, y_train)}\n')
print(f' R2 linreg en test: {linreg.score(X_test_sc, y_test)}\n')
print(f' R2 lasso en train: {lasso.score(X_train_sc, y_train)}\n')
print(f' R2 lasso en test: {lasso.score(X_test_sc, y_test)}\n')
print(f' R2 ridge en train: {ridge.score(X_train_sc, y_train)}\n')
print(f' R2 ridge en test: {ridge.score(X_test_sc, y_test)}\n')
print(f' R2 ElasticNet en train: {elastic.score(X_train_sc, y_train)}\n')
print(f' R2 ElasticNet en test: {elastic.score(X_test_sc, y_test)}\n')

### 2. Otros Modelos

#### 2.1 Decission Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
dt = DecisionTreeRegressor(criterion='squared_error')

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(50, 100))
plot_tree(dt, fontsize=10);

In [ ]:
dt.score(X_train, y_train), dt.score(X_test, y_test)

In [ ]:
mse(y_train, dt.predict(X_train)), mse(y_test, dt.predict(X_test)) 

In [ ]:
mse(y_train, dt.predict(X_train), squared=False), mse(y_test, dt.predict(X_test), squared=False) 

#### 2.2 Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_train, y_train), rf.score(X_test, y_test)

In [ ]:
mse(y_train, rf.predict(X_train)), mse(y_test, rf.predict(X_test)) 

In [ ]:
mse(y_train, rf.predict(X_train), squared=False), mse(y_test, rf.predict(X_test), squared=False) 

#### 2.3 SVR

In [ ]:
from sklearn.svm import SVR

In [ ]:
svr = SVR()

In [ ]:
svr.fit(X_train, y_train)

In [ ]:
svr.score(X_train, y_train), svr.score(X_test, y_test)

In [ ]:
mse(y_train, svr.predict(X_train)), mse(y_test, svr.predict(X_test)) 

In [ ]:
mse(y_train, svr.predict(X_train), squared=False), mse(y_test, svr.predict(X_test), squared=False) 